### 문제
- 왠진 모르겠지만 문제 이해가 참 어려웠다. 첫번째 나무 $a_1 = 1$ 이므로 첫번째로 자를 수 있는 나무는 $a_1$ 밖에 없다. \
해석하자면 전기톱 충전 비용이 $b_1$ 로 최대인 상태로 시작해서 모든 나무를 자르는데에 필요한 최소 충전 비용을 구하는 문제이다.
- 따라서 예제 1의 정답은 5 * 5 = 25 이다.

### 풀이
- 점화식을 다음과 같이 정의할 수 있다. 
  - $dp[i]$ = `i번째 나무까지 자르는데 필요한 최소 충전 비용`
- 나무가 완전히 잘리지 않다면 전기톱을 충전할 수 없으므로 초기값은 $dp[1] = b_1 * 1$ 이다.
- 한개의 나무를 모두 자르기 전까지 충전 비용이 변하지는 않으므로, 나무를 자를 때 자를 나무를 이것저것 바꾸는 것을 고려할 필요는 없다.
  - $i$ 번째 나무를 자르기로 결정했다면 바로 나무의 길이인 $a_i$ 만큼 자르면 된다는 의미이다. \
  즉, `이전까지 자른 최소 충전비용 = 선택했던 나무의 번호 중 최댓값의 충전비용 = x` 라고 할 때, $x \cdot a_i$ 만큼 더해주면 된다.
- 매번 i번째 나무를 골라서 기울기를 줄일 기회가 있으므로, 그 중 최솟값을 더해야 한다
- 따라서 최종적으로 $\displaystyle dp[i] = \min_{j<i}(dp[j] + a[i] \cdot b[j])$ 이 된다.
- 점화식이 Convex hull trick 을 이용한 최적화 점화식과 같으므로 이를 적용할 수 있다.
  - 또한 $a$ 는 단조증가 하므로 $\Omicron(N)$ 에 해결할 수 있다.

In [ ]:
import io, os
input=io.BytesIO(os.read(0, os.fstat(0).st_size)).readline

def cross(a: tuple[int, int], b: tuple[int, int]):
  return (b[1] - a[1]) / (a[0] - b[0])

def sol() :
  N = int(input())
  A = list(map(int, input().split()))
  B = list(map(int, input().split()))

  S = []
  DP = [0] * N
  for i in range(1, N):
    nl = [B[i-1], DP[i-1], 0] 
    while S : #새로운 직선을 넣기 전에 top과의 교점을 구해서 top의 시작점보다 교점이 왼쪽에 있으면 pop
      nl[2] = cross(S[-1], nl)
      if S[-1][2] < nl[2] : break
      S.pop()
    S.append(nl) #새로운 직선을 넣는다

    x = A[i] #현재 x좌표를 포함하는 선분을 이분탐색으로 찾는다.
    pos = len(S) - 1
    if x < S[-1][2] :
      lo, hi = 0, len(S) - 1
      while lo + 1 < hi :
        mid = (lo + hi) // 2
        if x < S[mid][2] : hi = mid
        else : lo = mid
      pos = lo
      
    DP[i] = S[pos][0] * x + S[pos][1]
  print(DP[-1])
 
sol()

- $\Omicron(N \log N)$ 풀이

In [ ]:
import io, os
input=io.BytesIO(os.read(0, os.fstat(0).st_size)).readline

def cross(a: tuple[int, int], b: tuple[int, int]):
  return (b[1] - a[1]) / (a[0] - b[0])

def sol() :
  N = int(input())
  A = list(map(int, input().split()))
  B = list(map(int, input().split()))

  S = []
  pos = 0 #현재 S에서 가장 위에 있는 직선의 인덱스
  DP = [0] * N
  for i in range(1, N):
    nl = [B[i-1], DP[i-1], 0] 
    while S : #새로운 직선을 넣기 전에 top과의 교점을 구해서 top의 시작점보다 교점이 왼쪽에 있으면 pop
      nl[2] = cross(S[-1], nl)
      if S[-1][2] < nl[2] : break
      # 이때 pos가 top이였으면 pos를 같이 감소한다.
      S.pop()
      if pos == len(S) : pos -= 1
    S.append(nl) #새로운 직선을 넣는다

    x = A[i]
    while pos < len(S) - 1 and S[pos+1][2] < x : #pos가 x좌표를 포함할 때 까지 pos를 증가시킨다.
      pos += 1

    DP[i] = S[pos][0] * x + S[pos][1]
  print(DP[-1])
 
sol()

- $\Omicron(N)$ 풀이

In [ ]:
import io, os
input=io.BytesIO(os.read(0, os.fstat(0).st_size)).readline

from bisect import bisect_left
class CHT :
  def __init__(self, A=[], B=[], EPS=1e-7) :
    self.A = A
    self.B = B
    self.I = []
    self.X = []
    self.i = 0
    self.EPS = EPS
    self.build()
  
  def intersect(self, i, j) :
    return (self.B[j] - self.B[i]) / (self.A[i] - self.A[j])

  def add(self, a, b, i=None) : #Amortized O(1)
    self.A.append(a)
    self.B.append(b)
    I, X = self.I, self.X
    if i is None :
      i = self.i
      self.i += 1

    while True:
      if not I:
        I.append(i)
        break
      elif (self.A[I[-1]] - self.A[i]) < self.EPS :
        if self.B[I[-1]] < self.B[i] :
          break
        I.pop()
        if X: X.pop()
      else:
        x = self.intersect(i, I[-1])
        if X and x <= X[-1] :
          I.pop()
          X.pop()
        else:
          I.append(i)
          X.append(x)
          break

  def build(self) :
    I, X = [], []    
    order = sorted(range(len(self.A)), key=self.A.__getitem__, reverse=True)
    for i in order:
      self.add(self.A[i], self.B[i], i)
    self.I = I
    self.X = X

  def query(self, x) : #O(logN)
    i = self.I[bisect_left(self.X, x + self.EPS)]
    return self.A[i] * x + self.B[i]

def sol() :
  N = int(input())
  LA = list(map(int, input().split()))
  LB = list(map(int, input().split()))

  DP = [0] * N
  cht = CHT()
  for i in range(1, N) :
    cht.add(LB[i-1], DP[i-1])
    DP[i] = cht.query(LA[i])
  
  print(DP[-1])

sol()

- 템플릿을 이용한 풀이. 템플릿의 적절한 변형이 있다.